In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os

In [16]:
#path for 11 models
path = "/home/mlopez/EXEC/tg_annual_rcp85/"

In [17]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [18]:
files

['/home/mlopez/EXEC/tg_annual_rcp85/ACCESS1-3_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/BNU-ESM_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/CanESM2_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/CMCC-CMS_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/GFDL-ESM2M_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/HadGEM2-CC_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/INM-CM4_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/IPSL-CM5A-LR_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/IPSL-CM5B-LR_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/MPI-ESM-LR_rcp85_tg_mean_annual.nc',
 '/home/mlopez/EXEC/tg_annual_rcp85/NorESM1-M_rcp85_tg_mean_annual.nc']

# Create DF of all models

In [19]:
dflist = []

In [20]:
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [21]:
dflist

[                                    tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  258.826111
                      1951-01-01  258.880676
                      1952-01-01  258.705017
                      1953-01-01  258.466431
                      1954-01-01  259.760803
 ...                                     ...
 40.041039 -74.045807 2096-01-01  289.731934
                      2097-01-01  289.322571
                      2098-01-01  290.335632
                      2099-01-01  289.968933
                      2100-01-01  290.880554
 
 [11507106 rows x 1 columns],                                     tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  257.977081
                      1951-01-01  256.822937
                      1952-01-01  259.196228
                      1953-01-01  258.807953
                      1954-01-01  257.551300
 ...                                     ...
 40.041039 -74.045807 20

In [22]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 1min 5s, sys: 5min 53s, total: 6min 59s
Wall time: 31.1 s


In [23]:
df11

tg_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  258.826111
                     1951-01-01  258.880676
                     1952-01-01  258.705017
                     1953-01-01  258.466431
                     1954-01-01  259.760803
...                                     ...
40.041039 -74.045807 2096-01-01  289.984100
                     2097-01-01  290.481293
                     2098-01-01  290.068420
                     2099-01-01  290.675110
                     2100-01-01  290.421906

[126578166 rows x 1 columns]

In [24]:
dftgm = df11.reset_index()

In [25]:
dftgm

,lat,lon,time,tg_mean
0,66.623306,-89.045212,1950-01-01,258.826111
1,66.623306,-89.045212,1951-01-01,258.880676
2,66.623306,-89.045212,1952-01-01,258.705017
3,66.623306,-89.045212,1953-01-01,258.466431
4,66.623306,-89.045212,1954-01-01,259.760803
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,289.984100
126578162,40.041039,-74.045807,2097-01-01,290.481293
126578163,40.041039,-74.045807,2098-01-01,290.068420
126578164,40.041039,-74.045807,2099-01-01,290.675110


# Get the mean for 11 models

In [26]:
#Takes 48s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 1min 6s, sys: 3min 37s, total: 4min 43s
Wall time: 1min 10s


In [27]:
dftgm2

tg_mean
lat       lon        time                  
40.041039 -89.045212 1950-01-01  284.527740
                     1951-01-01  284.319031
                     1952-01-01  284.219299
                     1953-01-01  284.076569
                     1954-01-01  284.236328
...                                     ...
66.623306 -61.296322 2096-01-01  270.468384
                     2097-01-01  270.841522
                     2098-01-01  270.916290
                     2099-01-01  270.619659
                     2100-01-01  271.236176

[11507106 rows x 1 columns]

# Transform temp to Celsius

In [28]:
dfC = dftgm2.copy()
dfC["tg_mean"] = dfC["tg_mean"] -273.15
    

# Define 3 periods of time: 

In [29]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [30]:
dfp.reset_index()

,time,lat,lon,tg_mean
0,0.0,40.041039,-89.045212,11.588665
1,0.0,40.041039,-88.961884,11.569731
2,0.0,40.041039,-88.878548,11.559522
3,0.0,40.041039,-88.795219,11.539219
4,0.0,40.041039,-88.711891,11.556223
...,...,...,...,...
228613,2.0,66.623306,-61.629639,-3.941922
228614,2.0,66.623306,-61.546310,-4.263534
228615,2.0,66.623306,-61.462978,-4.064792
228616,2.0,66.623306,-61.379650,-4.003588


# Period 2041-2071

In [31]:
df2050 = dfp.query("time==1.0")
df2050.reset_index()

,time,lat,lon,tg_mean
0,1.0,40.041039,-89.045212,14.524052
1,1.0,40.041039,-88.961884,14.506697
2,1.0,40.041039,-88.878548,14.497858
3,1.0,40.041039,-88.795219,14.477720
4,1.0,40.041039,-88.711891,14.497208
...,...,...,...,...
76201,1.0,66.623306,-61.629639,-6.319748
76202,1.0,66.623306,-61.546310,-6.631722
76203,1.0,66.623306,-61.462978,-6.425197
76204,1.0,66.623306,-61.379650,-6.355822


# Period 2071-2100

In [32]:
df2080 = dfp.query("time==2.0")
df2080.reset_index()

,time,lat,lon,tg_mean
0,2.0,40.041039,-89.045212,16.489874
1,2.0,40.041039,-88.961884,16.473457
2,2.0,40.041039,-88.878548,16.465921
3,2.0,40.041039,-88.795219,16.446129
4,2.0,40.041039,-88.711891,16.467459
...,...,...,...,...
76201,2.0,66.623306,-61.629639,-3.941922
76202,2.0,66.623306,-61.546310,-4.263534
76203,2.0,66.623306,-61.462978,-4.064792
76204,2.0,66.623306,-61.379650,-4.003588


## Import dataframe with polygons

In [33]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [34]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


# Merge data with mask 

## Period 2041-2071

In [35]:
dftg2050TG = pd.merge(df2050, dfpolyshape, on=["lat","lon"])
dftg2050TG

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,10.454852,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,10.429601,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,10.405848,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,10.391270,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,10.352727,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,0.235736,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,0.285438,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,0.099810,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,0.198478,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


## Period 2071-2100

In [36]:
dftg2080TG = pd.merge(df2080, dfpolyshape, on=["lat","lon"])
dftg2080TG

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,12.741622,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,12.717044,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,12.693558,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,12.679183,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,12.640734,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,2.868908,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,2.917686,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,2.736932,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,2.834848,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


# Create GeoJson for each territory

## Period 2041-2071

In [37]:
listTG2050 = []
for tg in dftg2050TG['TER_GUIDE'].unique().tolist():
    df = dftg2050TG[dftg2050TG.TER_GUIDE == tg]
    print (tg)
    listTG2050.append(df)
listTG2050

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon    tg_mean TER_GUIDE  \
 0     44.957512 -74.629120  10.454852        1a   
 1     44.957512 -74.545792  10.429601        1a   
 2     44.957512 -74.462463  10.405848        1a   
 3     44.957512 -74.379128  10.391270        1a   
 4     44.957512 -74.295799  10.352727        1a   
 ...         ...        ...        ...       ...   
 1236  46.207458 -72.629204   8.826795        1a   
 1345  46.290791 -72.879189   8.688724        1a   
 1347  46.290791 -72.795860   8.732653        1a   
 1349  46.290791 -72.712532   8.749599        1a   
 1351  46.290791 -72.629204   8.747013        1a   
 
                                                geometry  
 0     POLYGON ((-74.62590 44.99918, -74.58746 44.999...  
 1     POLYGON ((-74.58746 44.99876, -74.58746 44.999...  
 2     POLYGON ((-74.50413 44.99784, -74.50413 44.999...  
 3     POLYGON ((-74.42079 44.99530, -74.42079 44.999...  
 4     POLYGON ((-74.33746 44.99215, -74.33746 44.999...  
 ...                

In [38]:
for tg in listTG2050:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf2050 = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf2050.to_file(gdf2050['TER_GUIDE'].iloc[0]+"_2050_tg_mean_annual_rcp85_50.json", driver="GeoJSON")

## Period 2041-2071

In [39]:
listTG2080 = []
for tg in dftg2080TG['TER_GUIDE'].unique().tolist():
    df = dftg2080TG[dftg2080TG.TER_GUIDE == tg]
    print (tg)
    listTG2080.append(df)
listTG2080

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon    tg_mean TER_GUIDE  \
 0     44.957512 -74.629120  12.741622        1a   
 1     44.957512 -74.545792  12.717044        1a   
 2     44.957512 -74.462463  12.693558        1a   
 3     44.957512 -74.379128  12.679183        1a   
 4     44.957512 -74.295799  12.640734        1a   
 ...         ...        ...        ...       ...   
 1236  46.207458 -72.629204  11.207509        1a   
 1345  46.290791 -72.879189  11.075989        1a   
 1347  46.290791 -72.795860  11.119996        1a   
 1349  46.290791 -72.712532  11.136353        1a   
 1351  46.290791 -72.629204  11.132958        1a   
 
                                                geometry  
 0     POLYGON ((-74.62590 44.99918, -74.58746 44.999...  
 1     POLYGON ((-74.58746 44.99876, -74.58746 44.999...  
 2     POLYGON ((-74.50413 44.99784, -74.50413 44.999...  
 3     POLYGON ((-74.42079 44.99530, -74.42079 44.999...  
 4     POLYGON ((-74.33746 44.99215, -74.33746 44.999...  
 ...                

In [40]:
for tg in listTG2080:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf2080 = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf2080.to_file(gdf2080['TER_GUIDE'].iloc[0]+"_2080_tg_mean_annual_rcp85_50.json", driver="GeoJSON")

# ---- Create GeoJson file for the complete shape----

## Period 2041-2071

In [41]:
geometry = dftg2080TG["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg2050TG, crs=crs, geometry=geometry)
gdf.to_file("TG_2050_tg_mean_annual_rcp85_50.json", driver="GeoJSON")

NameError: name 'dftg11TG' is not defined

## Period 2071-2100

In [ ]:
geometry = dftg2080TG["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg2080TG, crs=crs, geometry=geometry)
gdf.to_file("TG_2080_tg_mean_annual_rcp85_50.json", driver="GeoJSON")